In [9]:
import cv2
import numpy as np

# 테스트를 위한 키보드 입력으로 받기
def chooseType(input_key):
    global frame_type
    if input_key==ord('a'):
        frame_type = 0
    elif input_key==ord('b'):
        frame_type = 1
    elif input_key==ord('c'):
        frame_type = 2
    elif input_key==ord('d'):
        frame_type = 3
    elif input_key==ord('e'):
        frame_type = 4
    elif input_key==ord('f'):
        frame_type = 5
    elif input_key==ord('t'):
        frame_type = 99
    return frame_type

# 프레임 스케치화
def frame_sketch(frame):
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.GaussianBlur(frame_gray, (9,9), 0)
    edges = cv2.Laplacian(frame_gray, -1, None, 5)
    ret, frame_sketch = cv2.threshold(edges, 70, 255, cv2.THRESH_BINARY_INV)
    frame_sketch = cv2.erode(frame_sketch, kernel)
    frame_paint = cv2.blur(frame, (10,10))
    frame_paint = cv2.bitwise_and(frame_paint, frame_paint, mask=frame_sketch)
    return frame_paint

# 프레임 좌우거울
def frame_mirrorh(frame):
    map_y, map_x = np.indices((rows, cols), dtype=np.float32)
    map_mirrorh_x, map_mirrorh_y = map_x.copy(), map_y.copy()
    map_mirrorh_x[: , cols//2:] = cols-map_mirrorh_x[: , cols//2:]-1
    mirrorh = cv2.remap(frame, map_mirrorh_x, map_mirrorh_y, cv2.INTER_LINEAR)
    return mirrorh

# 프레임 물결화
def frame_wave(frame):
    map_y, map_x = np.indices((rows, cols), dtype=np.float32)
    map_wave_x, map_wave_y = map_x.copy(), map_y.copy()
    map_wave_x = map_wave_x + 15*np.sin(map_y/20)
    map_wave_y = map_wave_y + 15*np.sin(map_x/20)
    wave = cv2.remap(frame,map_wave_x,map_wave_y,cv2.INTER_LINEAR,None,\
                     cv2.BORDER_REPLICATE)
    return wave

# 프레임 볼록렌즈
def frame_lenz(frame):
    map_y, map_x = np.indices((rows, cols), dtype=np.float32)
    map_lenz_x = 2*map_x/(cols-1)-1
    map_lenz_y = 2*map_y/(rows-1)-1
    
    r_convex, theta = cv2.cartToPolar(map_lenz_x, map_lenz_y)
    r_convex[r_convex<1] = r_convex[r_convex<1]**1.5
    
    map_convex_x, map_convex_y = cv2.polarToCart(r_convex, theta)
    map_convex_x = ((map_convex_x+1)*cols-1)/2
    map_convex_y = ((map_convex_y+1)*rows-1)/2
    convex = cv2.remap(frame, map_convex_x, map_convex_y, cv2.INTER_LINEAR)
    return convex

def countFinger(frame):
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    skin1 = np.array([0, 0, 10])
    skin2 = np.array([40, 255, 255])
    skin3 = np.array([170, 0, 10])
    skin4 = np.array([180, 255, 255])
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    frame_mask1 = cv2.inRange(frame, skin1, skin2)
    frame_mask2 = cv2.inRange(frame, skin3, skin4)
    frame1 = cv2.bitwise_and(frame, frame, mask=frame_mask1)
    frame2 = cv2.bitwise_and(frame, frame, mask=frame_mask2)
    frame = cv2.bitwise_or(frame1, frame2)
    frame = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frameCP = frame.copy()
    ret, frameCP = cv2.threshold(frame, 20, 255, cv2.THRESH_BINARY)
    frameCP =cv2.morphologyEx(frameCP, cv2.MORPH_CLOSE, k)
    ret, frameSS = cv2.threshold(frame, 20, 255, cv2.THRESH_BINARY)
    
    frameSS =cv2.morphologyEx(frameSS, cv2.MORPH_CLOSE, k)
    frameSS =cv2.morphologyEx(frameSS, cv2.MORPH_OPEN, k, iterations=13)
    contours, hierachy = cv2.findContours(frameSS, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    framePrint = frameSS.copy()
    
    framePrint = cv2.cvtColor(framePrint, cv2.COLOR_GRAY2BGR)
    fra = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    
    for c in contours:
        mmt = cv2.moments(c)
        cx = int(mmt['m10']/mmt['m00'])
        cy = int(mmt['m01']/mmt['m00'])
        #a = mmt['m00']
        
        (x,y), radius = cv2.minEnclosingCircle(c)
        radius *= 1.2
        
        count = 0
        isCount = 0
        for n in range(360):
            angle = n * np.pi / 180
            x = int(np.cos(angle)*radius)
            y = int(np.sin(angle)*radius)
            cv2.circle(framePrint, (cx+x,cy+y), 1, (0,255,255),-1)
            cv2.circle(fra, (cx+x,cy+y), 1, (0,255,255),-1)
        
            if cy+y<0 or cy+y>=rows:
                continue
            elif cx+x<0 or cx+x>=cols:
                continue
            
            if frameCP[cy+y][cx+x] == 0 and isCount == 1:
                isCount = 0
            elif frameCP[cy+y][cx+x] == 255 and isCount == 0:
                isCount = 1
                count += 1
                cv2.circle(framePrint, (cx+x,cy+y), 7, (0,0,255),-1)
                cv2.circle(fra, (cx+x,cy+y), 7, (0,0,255),-1)
        cv2.circle(framePrint, (cx,cy), 5, (0,255,255),-1)
        
    merged = np.hstack((fra, framePrint))
    return merged

# 메인함수
if __name__ == "__main__":
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    rows, cols = 480, 640
    frame_type = 0
    input_key = 0
    b_count = 0
    
    while True:
        # 프레임 읽기
        _, frame = cap.read()
        
        # 프레임 설정별로 다르게 출력
        if frame_type==1:
            frame = frame_sketch(frame)
        elif frame_type==2:
            frame = cv2.Laplacian(frame, -1)
        elif frame_type==3:
            frame = frame_mirrorh(frame)
        elif frame_type==4:
            frame = frame_wave(frame)
        elif frame_type==5:
            frame = frame_lenz(frame)
        elif frame_type==99:
            frame = countFinger(frame)
        
        cv2.imshow("VideoFrame",frame)
        # 사전 테스트를 위한 키보드 입력으로 조작
        input_key = cv2.waitKey(1)
        if input_key==27:
            break
        else:
            frame_type = chooseType(input_key)

    cap.release()
    cv2.destroyAllWindows()